In [1]:
# Importing Libs

import cv2
import numpy as np
from keras.models import load_model

2023-03-28 17:15:36.101532: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-28 17:15:36.285638: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-03-28 17:15:36.286916: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-28 17:15:37.459412: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [23]:
# Check for Camera availability

# List all the available camera devices
for i in range(4):
    cap = cv2.VideoCapture(i)
    if cap.isOpened():
        print(f"Camera {i} is available")
    else:
        print(f"Camera {i} is not available")


Camera 0 is available
Camera 1 is not available
Camera 2 is not available
Camera 3 is not available


[ WARN:0@1644.519] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video1): can't open camera by index
[ERROR:0@1644.520] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range
[ WARN:0@1644.520] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video2): can't open camera by index
[ERROR:0@1644.521] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range
[ WARN:0@1644.521] global cap_v4l.cpp:982 open VIDEOIO(V4L2:/dev/video3): can't open camera by index
[ERROR:0@1644.522] global obsensor_uvc_stream_channel.cpp:156 getStreamChannelGroup Camera index out of range


In [2]:

# Filter out the background noise from the image

def filter_background_noise(frame):
    # Convert the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to smooth the image and remove noise
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Apply adaptive thresholding to create a binary image
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

    # Find contours in the binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Loop over the contours and draw them on the original image
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 750:  # adjust this threshold to filter out smaller contours
            cv2.drawContours(frame, [contour], 0, (0, 0, 0), 2)
    
    return frame



In [6]:
# Preprocess the image from camera before classification

def preprocess_image(frame):

    # Convert the image to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Resize the image to (64, 64) and convert it to RGB format
    resized = cv2.resize(gray, (64, 64))
    img = cv2.cvtColor(resized, cv2.COLOR_GRAY2RGB)

    # Normalize the image
    normalized = img / 255.0



    # Reshape the image
    reshaped = np.reshape(normalized, (64, 64, 3))

    return reshaped



In [7]:
import cv2
import numpy as np
from keras.models import load_model

# Load the Indian Sign Language model
model = load_model('isl_model.h5')

# Create a dictionary of signs and their corresponding labels
signs = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M',
         13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y'}

# Function to predict the sign from webcam input
def predict_sign():
    # Open the webcam
    cap = cv2.VideoCapture(0)

    while True:
        # Read the frame from the webcam
        ret, frame = cap.read()

        # preprocessing the frame before classification
        preprocessed_frame = preprocess_image(frame)

        # Make the prediction using the model
        # prediction = model.predict(preprocessed_frame)
        # predicted_class = np.argmax(prediction)

        # Get the predicted sign label
        # sign = signs[predicted_class]

        # Display the predicted sign label on the screen
        # cv2.putText(frame, sign, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Indian Sign Language Recognition', preprocessed_frame)

        # Stop the program when the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close all windows
    cap.release()
    cv2.destroyAllWindows()

# Call the function to start predicting signs from webcam input
predict_sign()


In [ ]:
import os
import numpy as np

data_path = '' # Replace with the path to your ISL dataset directory

# Get the list of all directories in the dataset directory
dirs = os.listdir(data_path)

# Sort the directories alphabetically for consistency
dirs.sort()

# Create a dictionary to map each directory name to a label
label_map = {dirs[i]: i for i in range(len(dirs))}

# Initialize empty lists for images and labels
images = []
labels = []

# Loop through all directories in the dataset directory
for dir_name in dirs:
    # Get the label for the current directory
    label = label_map[dir_name]
    
    # Get the path to the current directory
    dir_path = os.path.join(data_path, dir_name)
    
    # Loop through all files in the current directory
    for filename in os.listdir(dir_path):
        # Load the image file using your preferred image processing library
        # Here we assume you are using OpenCV
        image = cv2.imread(os.path.join(dir_path, filename))
        
        # Preprocess the image as needed (e.g. resize, normalize, etc.)
        
        # Append the preprocessed image and label to the lists
        images.append(preprocessed_image)
        labels.append(label)

# Convert the lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

# Print the shape of the images and labels arrays
print('Images shape:', images.shape)
print('Labels shape:', labels.shape)


In [ ]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import np_utils

In [ ]:
# Define the path to the dataset folder
dataset_path = './Indian/'

In [ ]:
# Define the list of classes
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# Define the input shape of the images
input_shape = (128, 128, 3)

# Define the number of classes
num_classes = len(classes)

# Define the batch size
batch_size = 32


In [ ]:
# Load the dataset
X = []
y = []
for i, cls in enumerate(classes):
    cls_path = os.path.join(dataset_path, cls)
    for file in os.listdir(cls_path):
        if file.endswith('.jpg'):
            img_path = os.path.join(cls_path, file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, input_shape[:2])
            X.append(img)
            y.append(i)

In [ ]:
# Convert the input and target data to numpy arrays
X = np.array(X)
y = np.array(y)

In [ ]:
from skimage.transform import resize

In [ ]:
# Resize the images to 64x64
X_resized = np.array([resize(img, (64, 64)) for img in X])

In [ ]:
# Normalize the input data
X = X_resized.astype('float32') / 255.0

# One-hot encode the target data
y = np_utils.to_categorical(y, num_classes)


In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_test, y_test))

In [ ]:
import os
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import np_utils

# Define the path to the dataset folder
dataset_path = 'Sign Language//Indian'

# Define the list of classes
classes = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '1', '2', '3', '4', '5', '6', '7', '8', '9']

# Define the input shape of the images
input_shape = (128, 128, 3)

# Define the number of classes
num_classes = len(classes)

# Define the batch size
batch_size = 32

# Load the dataset
X = []
y = []
for i, cls in enumerate(classes):
    cls_path = os.path.join(dataset_path, cls)
    for file in os.listdir(cls_path):
        if file.endswith('.jpg'):
            img_path = os.path.join(cls_path, file)
            img = cv2.imread(img_path)
            img = cv2.resize(img, input_shape[:2])
            X.append(img)
            y.append(i)

# Convert the input and target data to numpy arrays
X = np.array(X)
y = np.array(y)

# Normalize the input data
X = X.astype('float32') / 255.0

# One-hot encode the target data
y = np_utils.to_categorical(y, num_classes)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_test, y_test))

# Save the trained model
model.save('isl_model.h5')


In [ ]:
import numpy as np
import os
import cv2
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.utils import np_utils

# Define the data directory
data_dir = './Indian/'

# Define the categories
categories = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
              '1', '2', '3', '4', '5', '6', '7', '8', '9']

# Define the image size
img_size = 64

# Create an empty list to store the data and labels
data = []
labels = []

# Loop through the categories and load the data
for category in categories:
    path = os.path.join(data_dir, category)
    class_num = categories.index(category)
    for img in os.listdir(path):
        img_arr = cv2.imread(os.path.join(path, img))
        img_arr = cv2.resize(img_arr, (img_size, img_size))
        data.append(img_arr)
        labels.append(class_num)

# Convert the data and labels to NumPy arrays
data = np.array(data)
labels = np.array(labels)

# Normalize the input data
X = data.astype('float32') / 255.0

# One-hot encode the target data
y = np_utils.to_categorical(labels, num_classes=len(categories))

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the CNN model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_size, img_size, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(categories), activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
batch_size = 128
model.fit(X_train, y_train, batch_size=batch_size, epochs=10, validation_data=(X_test, y_test))

# Save the trained model
model.save('isl_model.h5')


In [ ]:
# Save the trained model
model.save('isl_model.h5')